### importing modules

In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import time
import traceback
import openai
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

### long Ans Function

In [5]:
def longAnsFunc():
    longAnsDf = pd.read_csv('longAns.csv')
    longAnsList = longAnsDf['Answer']
    longAnsList.dropna(inplace=True)
    return longAnsList

### dotenv

In [6]:
load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

### main

### generate analogy

In [7]:
def rephrasing_style_three_four(answer: str) -> str:

    prompts_dict = {
        'prompt_analogy' : (
            "You are an AI specializing in creating relatable analogies. "
            "Given the following explanation, provide a short, real-world analogy that is "
            "easy to understand and relatable for most people.\n\n"
            f"Answer: {answer}\n\n"
            "Analogy:"
            "Ensure that the Analogy should not exceed more than 50 words."
        ),
        
        'prompt_example' : (
            "You are an Educational AI assistant who excels in Bachelors of Technology course. You have all the knowledge about every subject and it's whole content."
            "Given the following answer, generate three concise examples that relate to the content. Each example must be brief and to the point."
            f"Answer: {answer} \n\n"
            "Ensure that the word length of each example should not exceed more than 15 words.\n"
        ),
    }

    # Define the dictionary of models
    models_dict = {
        "model_analogy": ChatOpenAI(
                model="gpt-4o-mini",
                temperature=0.7,
                n=1,
                stop=['\n'],
        ),
        
        "model_example": ChatOpenAI(
                model="gpt-4o-mini",
                temperature=0.7,
                n=3,
        ),
        
        
    }

    prompt = prompts_dict['prompt_example']
    model = models_dict['model_example']



    try:
        start_time = time.time()
        message = prompt.format(answer = answer)
        response = model.invoke(message)
        end_time = time.time()
        print(end_time-start_time)

        analogy = response.content
        return analogy

    except Exception as e:
        return f"An error occurred: {e}"

### ask want to save csv

In [8]:
def ask_want_to_save_csv(final_df):
    import re
    choice = input("Do you want to save the output csv (y/n) : ")
    if choice.lower() == 'y':
        output_dir = 'EXAMPLE_SAMPLE_OUTPUTS'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        fileName = input("Enter csv file name : ")
        valid_fileName = re.sub(r'[<>:"/\\|?*]', '_', fileName)
        save_path = os.path.join(output_dir, f'{valid_fileName}.csv')
        final_df.to_csv(save_path, index=True)
        print("CSV SAVED")
    else:
        print("Ok no problem")
        print(final_df.head())

### main

In [30]:
def main():
    longAnsList = longAnsFunc()
    starting_ind = int(input("Enter starting ques : ")) -1 
    ending_ind = int(input("Enter ending ques : ")) -1 
    longAnsList = longAnsList[starting_ind: ending_ind+1]
    #
    analogyList = []
    for longAns in longAnsList:
        analogy = generate_analogy(longAns)
        analogyList.append(analogy)

    final_df = pd.DataFrame(
        {
            'longAns' : longAnsList,
            'analogy' : analogyList,
        }
    )
    ask_want_to_save_csv(final_df)

if __name__ == '__main__':
    main()

Enter starting ques :  1
Enter ending ques :  1


1.6134049892425537


Do you want to save the output csv (y/n) :  n


Ok no problem
                                             longAns  \
0  An entrepreneur is an owner or founder of a bu...   

                                             analogy  
0  1. A tech startup founder develops an app to s...  


# Testing - 2

In [9]:
import json
import pandas as pd
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
import getpass
import time
import traceback

In [10]:
def parse_Response(response):
   if isinstance(response, str):
       # print(response)
       start_index = response.find("{")
       end_index = response.rfind("}")
       if start_index != -1 and end_index != -1:
           valid_json_content = response[start_index : end_index + 1]
           try:
               JSON_response = json.loads(valid_json_content.replace("\n", ""))
               # append_list_to_file(JSON_response)
               return JSON_response
           except json.JSONDecodeError as e:
               print(f"Error decoding JSON response: {e.__class__.__name__} - {e}\n\n Still trying to work on particular exceptions ...")
               print("Actual Content: ", valid_json_content)
       else:
           print("No valid JSON content found in the response.")
       # time.sleep(50)
   elif isinstance(response, dict):
       return response
   else:
       print("No response message found", type(response))

### getting summary

In [11]:
def summarizedAnsFunc():
    """
    Loads summarized answers from the 'genSum.csv' file and allows the user
    to select a valid range of questions for rephrasing.

    Returns:
        list : A list of summarized answers within the specified range
    """
    import ast

    try:
        # long answer
        longAnsList = longAnsFunc()
        
        # Load the data summary
        df = pd.read_csv('genSum.csv')
        df['summarized_ans'] = df['summary'].apply(ast.literal_eval)
        summarizedAnswerList = [i['summarizedAnswer'] for i in df.summarized_ans]

        # Validating dataset size
        total_questions = len(summarizedAnswerList)
        print(f"Enter the range of questions (1-{total_questions}) from the dataset you want to REPHRASE")

        # getting the user input with validation
        while True:
            try:
                starting_ind = int(input(f"Starting question index (1-{total_questions}): ")) - 1
                ending_ind = int(input(f"Ending question index (1-{total_questions}): ")) - 1

                if not (0 <= starting_ind < total_questions and 0 <= ending_ind < total_questions):
                    print(f"Indices must be between 1 and {total_questions}. Please try again.")
                elif starting_ind > ending_ind:
                    print("Starting index cannot be greater than the ending index. Please try again.")
                else:
                    break
            except ValueError:
                print("Invalid input. Please enter numeric value only.")

        #loop ended successfully , meaning the range entered is correct
        return summarizedAnswerList[starting_ind: ending_ind + 1], longAnsList[starting_ind: ending_ind+1]

    except FileNotFoundError:
        print("The file was not found. Please ensure it exists in the current directory.")
    except KeyError as e:
        print(f"Error : Missing column {e} in the dataset. Please check the file format.")
    except Exception as e:
        print(f"An unexpected error occurred in the summarizedAnsFunc() : {e}")


In [14]:
def main():
    summarizedAnsList, longAnsList = summarizedAnsFunc()

    #
    analogyList = []
    for longAns in longAnsList:
        analogy = generate_analogy(longAns)
        analogyList.append(analogy)


    for summ, analogy in zip(summarizedAnsList, analogyList):
        summary = parse_Response(summ)
        print(analogy)
        summary['summarized_answer'] = summary['summarized_answer'] + analogy
        print('\n\nNew summary')
        print(summary)

if __name__ == '__main__':
    main()

Enter the range of questions (1-18) from the dataset you want to REPHRASE


Starting question index (1-18):  1
Ending question index (1-18):  1


2.0343379974365234
1. A tech startup founder develops an app that simplifies online shopping experiences.  
2. An entrepreneur launches a sustainable fashion line to address environmental concerns in clothing.  
3. A restaurant owner pivots to delivery services during a pandemic to meet customer needs.  


New summary
{'summarized_answer': 'An entrepreneur is a business founder who identifies opportunities, assumes risks, and efficiently uses resources to generate profit and contribute to economic growth. Key characteristics include: \n\n- **Innovation**: Introducing new or improved products/services.\n- **Risk-taking**: Willingness to accept financial, professional, and personal risks.\n- **Vision and Goal Orientation**: Clear long-term goals drive decision-making.\n- **Resilience**: Overcoming hurdles and staying motivated.\n- **Self-Confidence**: Belief in achieving goals and mobilizing resources.\n- **Flexibility**: Adapting strategies to changing market conditions.\n- **Leadership